# Analysis

In [8]:
# Model code
from model import Param, Trial

# Other dependencies
import time
import itertools
from IPython.display import display
import pandas as pd
import plotly.express as px

In [6]:
# Run a single trial
single_trial = Trial()
single_trial.run_trial()

# Preview results
display(single_trial.patient_results_df)
display(single_trial.trial_results_df)
display(single_trial.interval_audit_df)

# Plot interval audit utilisation
fig = px.line(single_trial.interval_audit_df,
              x='simulation_time', y='perc_utilisation', color='run')
fig.show()

# Calculate and plot median utilisation
interval_audits_median = (single_trial
                          .interval_audit_df
                          .drop('resource_name', axis=1)
                          .groupby('simulation_time')
                          .median()
                          .reset_index())
fig = px.line(interval_audits_median,
              x='simulation_time',
              y='perc_utilisation')
fig.show()


,patient_id,q_time_nurse,time_with_nurse,run
0,1,0.000000,28.108654,0
1,2,0.000000,13.370060,0
2,3,0.000000,12.746160,0
3,4,0.000000,18.531603,0
4,5,0.000000,97.593265,0
...,...,...,...,...
579,115,24.655298,55.491892,4
580,116,27.673156,28.124405,4
581,117,26.613803,12.797927,4
582,118,27.101624,66.409634,4


,run_number,scenario,arrivals,mean_q_time_nurse,average_nurse_utilisation
0,0,0,115,0.260844,0.715650
1,1,0,111,4.667636,0.681010
2,2,0,115,5.412314,0.724797
3,3,0,124,2.885792,0.715003
4,4,0,119,19.915133,0.862291


,resource_name,simulation_time,number_utilised,number_available,queue_length,run,perc_utilisation
0,nurse,0,0,9,0,0,0.000000
1,nurse,5,1,9,0,0,0.111111
2,nurse,10,1,9,0,0,0.111111
3,nurse,15,1,9,0,0,0.111111
4,nurse,20,2,9,0,0,0.222222
...,...,...,...,...,...,...,...
595,nurse,575,9,9,7,4,1.000000
596,nurse,580,9,9,6,4,1.000000
597,nurse,585,9,9,5,4,1.000000
598,nurse,590,9,9,4,4,1.000000


In [9]:
# Run with 1 to 14 cores
speed = []
Param.number_of_runs = 100
for i in range(1, 15, 1):
    start_time = time.time()
    my_trial = Trial()
    my_trial.run_trial(cores=i)
    run_time = round((time.time() - start_time), 3)
    speed.append({'Cores': i, 'Run Time (seconds)': run_time})

# Display and plot time by number of cores
timing_results = pd.DataFrame(speed)
print(timing_results)
fig = px.line(timing_results, x='Cores', y='Run Time (seconds)')
fig.show()

    Cores  Run Time (seconds)
0       1               0.183
1       2               0.581
2       3               0.435
3       4               0.435
4       5               0.453
5       6               0.454
6       7               0.501
7       8               0.084
8       9               0.185
9      10               0.163
10     11               0.584
11     12               0.064
12     13               0.065
13     14               0.147


In [10]:
# TODO: Issue with this set-up is that you could unknowingly change g somewhere
# So here, for example, having to set g.number_of_runs back to a lower number
# Is there an alternative way of doing this that avoids that issue?
# For example, having those as inputs to Model() instead?

# Define a set of scenarios
Param.number_of_runs = 5
scenarios = {
    'patient_inter': [5, 10, 15],
    'mean_n_consult_time': [15, 20, 35],
    'number_of_nurses': [3, 6, 9]
}

# Find every possible permutation of the scenarios
all_scenarios_tuples = list(itertools.product(*scenarios.values()))
# Convert back into dictionaries
all_scenarios_dicts = [
    dict(zip(scenarios.keys(), p)) for p in all_scenarios_tuples]
# Preview some of the scenarios
print(f'There are {len(all_scenarios_dicts)} scenarios. For example:')
display(all_scenarios_dicts[0:6])

# Run the scenarios...
results = []
for index, scenario_to_run in enumerate(all_scenarios_dicts):
    # Overwrite defaults from the passed dictionary
    Param.scenario_name = index
    for key in scenario_to_run:
        setattr(Param, key, scenario_to_run[key])
    # Run trial and keep trial-level results
    my_trial = Trial()
    my_trial.run_trial()
    results.append(my_trial.trial_results_df)
# View mean results by scenario
display(pd.concat(results)
        .drop('run_number', axis=1)
        .groupby('scenario')
        .mean()
        .head(20))

# TODO: Issue: warm-up patients use resources but their activity is excluded
# from metrics. Post-warm-up patients queue behind these, making it look
# like resources are under-utilised during the measurement period if there are
# long queues (e.g. due to really short inter-arrival times)

There are 27 scenarios. For example:


[{'patient_inter': 5, 'mean_n_consult_time': 15, 'number_of_nurses': 3},
 {'patient_inter': 5, 'mean_n_consult_time': 15, 'number_of_nurses': 6},
 {'patient_inter': 5, 'mean_n_consult_time': 15, 'number_of_nurses': 9},
 {'patient_inter': 5, 'mean_n_consult_time': 20, 'number_of_nurses': 3},
 {'patient_inter': 5, 'mean_n_consult_time': 20, 'number_of_nurses': 6},
 {'patient_inter': 5, 'mean_n_consult_time': 20, 'number_of_nurses': 9}]

,arrivals,mean_q_time_nurse,average_nurse_utilisation
scenario,,,
0,107.4,28.752656,0.895605
1,119.2,0.650960,0.506919
2,119.6,0.004197,0.338975
3,87.0,76.038860,0.974331
4,118.8,2.880780,0.662758
5,119.6,0.105159,0.446922
6,51.0,172.464534,0.982788
7,98.8,42.185221,0.941436
8,116.8,6.628344,0.739750
